In [5]:
"""
Complete Agentic AI System with RAG - Using LCEL
This avoids all import issues by using modern LangChain Expression Language
"""

from langchain_core.tools import Tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain_community.chat_models import ChatOllama
import os
from dotenv import load_dotenv
import math
import re

load_dotenv()

def get_llm():
    """Initialize LLM based on environment configuration"""
    llm_type = os.getenv("LLM_TYPE", "groq")
    if llm_type == "groq":
        return ChatGroq(
            model="llama-3.1-8b-instant",
            temperature=0,  # Lower temp for better tool following
            max_tokens=1000,
            groq_api_key=os.getenv("GROQ_API_KEY")
        )
    else:
        return ChatOllama(model="llama3.1", temperature=0)

# ==============================================================================
# STEP 1: Create Knowledge Base for RAG Tool
# ==============================================================================

print("=" * 80)
print("BUILDING AGENTIC AI SYSTEM WITH LCEL")
print("=" * 80)

# Create a knowledge base about different topics
knowledge_docs = [
    Document(
        page_content="""
        Python Programming: Python is a high-level programming language known for 
        its simplicity and readability. Key features include dynamic typing, 
        automatic memory management, and extensive standard library. Popular for 
        data science, web development, and AI/ML applications.
        """,
        metadata={"topic": "programming", "language": "python"}
    ),
    Document(
        page_content="""
        Machine Learning Basics: Machine learning is a method of data analysis 
        that automates analytical model building. It uses algorithms that 
        iteratively learn from data. Types include supervised learning 
        (classification, regression), unsupervised learning (clustering), and 
        reinforcement learning (rewards-based).
        """,
        metadata={"topic": "AI", "subtopic": "ML"}
    ),
    Document(
        page_content="""
        Data Structures: Common data structures include arrays, linked lists, 
        stacks, queues, trees, and graphs. Arrays provide O(1) access time but 
        O(n) insertion. Linked lists offer O(1) insertion but O(n) access. 
        Hash tables provide O(1) average case for both operations.
        """,
        metadata={"topic": "computer_science", "subtopic": "data_structures"}
    ),
    Document(
        page_content="""
        Neural Networks: Artificial neural networks are computing systems inspired 
        by biological neural networks. They consist of layers of interconnected 
        nodes (neurons). Each connection has a weight adjusted during training. 
        Common architectures include feedforward, convolutional (CNN), and 
        recurrent (RNN) networks.
        """,
        metadata={"topic": "AI", "subtopic": "deep_learning"}
    )
]

# Setup embeddings and vector store
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

vectorstore = Chroma.from_documents(
    documents=knowledge_docs,
    embedding=embeddings,
    collection_name="agent_knowledge_base"
)

print("✓ Knowledge base created with", len(knowledge_docs), "documents")

# ==============================================================================
# STEP 2: Define Tools for the Agent
# ==============================================================================

# Tool 1: Calculator
def calculator(expression: str) -> str:
    """Evaluates mathematical expressions safely"""
    try:
        allowed_names = {
            'abs': abs, 'round': round, 'min': min, 'max': max,
            'sum': sum, 'pow': pow, 'sqrt': math.sqrt,
            'sin': math.sin, 'cos': math.cos, 'tan': math.tan,
            'pi': math.pi, 'e': math.e
        }
        result = eval(expression, {"__builtins__": {}}, allowed_names)
        return f"The result is: {result}"
    except Exception as e:
        return f"Error calculating: {str(e)}"

# Tool 2: Knowledge Base Retriever (RAG)
def knowledge_retriever(query: str) -> str:
    """Retrieves relevant information from the knowledge base"""
    retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
    docs = retriever.invoke(query)
    
    if not docs:
        return "No relevant information found in the knowledge base."
    
    context = "\n\n".join([f"Source {i+1}:\n{doc.page_content}" 
                           for i, doc in enumerate(docs)])
    return context

# Tool 3: String Analyzer
def string_analyzer(text: str) -> str:
    """Analyzes text and returns statistics"""
    words = text.split()
    unique_words = set(words)
    
    analysis = f"""Text Analysis:
- Character count: {len(text)}
- Word count: {len(words)}
- Unique words: {len(unique_words)}
- Average word length: {sum(len(w) for w in words) / len(words) if words else 0:.2f}
- Sentences (approx): {text.count('.') + text.count('!') + text.count('?')}"""
    return analysis

# Create tool definitions
tools = {
    "Calculator": {
        "func": calculator,
        "description": "Use for mathematical calculations. Input: valid math expression like '2+2' or 'sqrt(16)'"
    },
    "KnowledgeBase": {
        "func": knowledge_retriever,
        "description": "Search for information about programming, AI, ML, data structures, neural networks"
    },
    "TextAnalyzer": {
        "func": string_analyzer,
        "description": "Analyze text for word count, character count, statistics. Input: text to analyze"
    }
}

print("\n✓ Created", len(tools), "tools:")
for tool_name in tools.keys():
    print(f"  - {tool_name}")

# ==============================================================================
# STEP 3: Build Agent Using LCEL
# ==============================================================================

print("\n" + "=" * 80)
print("BUILDING AGENT WITH LCEL")
print("=" * 80)

# Create tool descriptions for prompt
tool_descriptions = "\n".join([
    f"- {name}: {info['description']}"
    for name, info in tools.items()
])

# Agent prompt template
agent_prompt = ChatPromptTemplate.from_template("""You are a helpful AI agent with access to tools. 

Available Tools:
{tool_descriptions}

To use a tool, respond in this exact format:
TOOL: ToolName
INPUT: input for the tool

If you don't need a tool, just provide your answer directly.

User Question: {question}

Your Response:""")

def parse_and_execute_tool(response: str) -> dict:
    """Parse LLM response and execute tool if needed"""
    # Check if response contains tool usage
    tool_match = re.search(r'TOOL:\s*(\w+)', response)
    input_match = re.search(r'INPUT:\s*(.+?)(?=\n|$)', response, re.DOTALL)
    
    if tool_match and input_match:
        tool_name = tool_match.group(1)
        tool_input = input_match.group(1).strip()
        
        if tool_name in tools:
            tool_output = tools[tool_name]["func"](tool_input)
            return {
                "used_tool": True,
                "tool_name": tool_name,
                "tool_input": tool_input,
                "tool_output": tool_output,
                "original_response": response
            }
    
    return {
        "used_tool": False,
        "response": response
    }

def agent_loop(question: str, max_iterations: int = 3, verbose: bool = True):
    """
    Main agent loop using LCEL components
    """
    llm = get_llm()
    conversation_history = ""
    
    for iteration in range(max_iterations):
        if verbose:
            print(f"\n{'='*80}")
            print(f"Iteration {iteration + 1}")
            print(f"{'='*80}")
        
        # Format prompt with history
        full_question = f"{question}\n\nPrevious steps:\n{conversation_history}" if conversation_history else question
        
        # Create chain
        chain = (
            {
                "question": RunnablePassthrough(),
                "tool_descriptions": lambda x: tool_descriptions
            }
            | agent_prompt
            | llm
            | StrOutputParser()
        )
        
        # Get response
        response = chain.invoke(full_question)
        
        if verbose:
            print(f"\nAgent Response:\n{response}")
        
        # Parse and execute tool
        result = parse_and_execute_tool(response)
        
        if result["used_tool"]:
            tool_name = result["tool_name"]
            tool_output = result["tool_output"]
            
            if verbose:
                print(f"\n→ Used Tool: {tool_name}")
                print(f"→ Tool Output: {tool_output}")
            
            # Add to history
            conversation_history += f"\nStep {iteration + 1}:"
            conversation_history += f"\n- Used {tool_name}"
            conversation_history += f"\n- Result: {tool_output}\n"
            
            # Check if we need another iteration
            if "final answer" in response.lower() or iteration == max_iterations - 1:
                # Generate final answer with tool results
                final_prompt = f"""Based on this information:
{conversation_history}

Provide a final answer to: {question}

Final Answer:"""
                final_response = llm.invoke(final_prompt)
                final_answer = final_response.content if hasattr(final_response, 'content') else str(final_response)
                
                if verbose:
                    print(f"\n{'='*80}")
                    print("FINAL ANSWER")
                    print(f"{'='*80}")
                    print(final_answer)
                
                return final_answer
        else:
            # No tool used, this is the final answer
            return result["response"]
    
    return "Agent reached max iterations"

# ==============================================================================
# STEP 4: Test the Agent
# ==============================================================================

print("\n" + "=" * 80)
print("TESTING AGENT")
print("=" * 80)

# Check for API key
if not os.getenv("GROQ_API_KEY"):
    print("\n⚠️  WARNING: No GROQ_API_KEY found!")
    print("Set GROQ_API_KEY in your .env file to test the agent")
    print("\nDemonstrating manual tool usage instead...\n")
    
    # Manual tool demonstrations
    print("=" * 80)
    print("MANUAL TOOL DEMONSTRATION")
    print("=" * 80)
    
    print("\n1. Calculator Tool:")
    print("   Input: 'sqrt(144) + 10 * 2'")
    result = calculator("sqrt(144) + 10 * 2")
    print(f"   Output: {result}")
    
    print("\n2. Knowledge Base Tool:")
    print("   Input: 'What is machine learning?'")
    result = knowledge_retriever("What is machine learning?")
    print(f"   Output:\n{result[:200]}...")
    
    print("\n3. Text Analyzer Tool:")
    sample_text = "Artificial intelligence is transforming the world."
    print(f"   Input: '{sample_text}'")
    result = string_analyzer(sample_text)
    print(f"   Output:\n{result}")

else:
    # Test with real agent
    test_questions = [
        "Calculate the square root of 144",
        "What is machine learning?",
        "Calculate 5 * 8 and then tell me about neural networks"
    ]
    
    for question in test_questions:
        print(f"\n{'='*80}")
        print(f"Question: {question}")
        print(f"{'='*80}")
        
        try:
            answer = agent_loop(question, max_iterations=3, verbose=True)
            print(f"\n✓ Complete")
        except Exception as e:
            print(f"Error: {e}")

# ==============================================================================
# STEP 5: Simple Agent Function (Easy to Use)
# ==============================================================================

print("\n" + "=" * 80)
print("SIMPLIFIED AGENT FUNCTION")
print("=" * 80)

def simple_agent(question: str, verbose: bool = False):
    """
    Simplified agent function - easy to use
    
    Usage:
        answer = simple_agent("Calculate sqrt(256)")
        answer = simple_agent("What is Python?")
    """
    return agent_loop(question, max_iterations=3, verbose=verbose)

print("""
Usage Example:

    # Simple usage
    answer = simple_agent("Calculate 15 * 8")
    print(answer)
    
    # With verbose output
    answer = simple_agent("What is machine learning?", verbose=True)
    print(answer)
    
    # Complex query
    answer = simple_agent("Calculate sqrt(144) and explain what neural networks are")
    print(answer)
""")

# ==============================================================================
# STEP 6: Architecture Diagram
# ==============================================================================

print("\n" + "=" * 80)
print("AGENT ARCHITECTURE (LCEL-Based)")
print("=" * 80)

print("""
┌─────────────────────────────────────────────────────────────┐
│                     USER QUESTION                           │
└───────────────────────┬─────────────────────────────────────┘
                        │
                        ▼
┌─────────────────────────────────────────────────────────────┐
│                  AGENT LOOP (LCEL)                          │
│  ┌──────────────────────────────────────────────┐           │
│  │  1. Format Prompt with Tool Descriptions     │           │
│  └────────────┬─────────────────────────────────┘           │
│               │                                              │
│               ▼                                              │
│  ┌──────────────────────────────────────────────┐           │
│  │  2. LLM Generates Response                   │           │
│  │     (May request tool usage)                 │           │
│  └────────────┬─────────────────────────────────┘           │
│               │                                              │
│               ▼                                              │
│  ┌──────────────────────────────────────────────┐           │
│  │  3. Parse Response                           │           │
│  │     - Tool needed? → Execute tool            │           │
│  │     - No tool? → Return answer               │           │
│  └────────────┬─────────────────────────────────┘           │
│               │                                              │
│               ▼                                              │
│  ┌──────────────────────────────────────────────┐           │
│  │  4. If tool used, add to history and loop    │           │
│  └──────────────────────────────────────────────┘           │
└─────────────────────────────────────────────────────────────┘
                        │
                        ▼
        ┌───────────────────────────┐
        │    AVAILABLE TOOLS        │
        ├───────────────────────────┤
        │  • Calculator             │
        │  • KnowledgeBase (RAG)    │
        │  • TextAnalyzer           │
        └───────────────────────────┘
                        │
                        ▼
┌─────────────────────────────────────────────────────────────┐
│                    FINAL ANSWER                             │
└─────────────────────────────────────────────────────────────┘
""")

# ==============================================================================
# Summary
# ==============================================================================

print("\n" + "=" * 80)
print("AGENTIC AI SYSTEM SUMMARY")
print("=" * 80)
print(f"""
✓ Created {len(tools)} tools (Calculator, KnowledgeBase, TextAnalyzer)
✓ Built agent using LCEL (no import issues!)
✓ Implemented ReAct-style reasoning loop
✓ RAG retrieval integrated as a tool

Key Features:
• No deprecated imports - all modern LCEL
• Simple tool execution system
• Iterative reasoning with max iterations
• Verbose mode for debugging
• Easy-to-use simple_agent() function

Usage:
    answer = simple_agent("Your question here")
    answer = simple_agent("Calculate sqrt(256) and explain ML", verbose=True)

Benefits of LCEL Approach:
• No AgentExecutor import issues
• More control over agent behavior
• Easier to debug and customize
• Works with latest LangChain versions
• Simpler, more transparent code

Status: {'✓ Ready to use!' if os.getenv('GROQ_API_KEY') else '⚠️  Set GROQ_API_KEY to activate'}
""")

BUILDING AGENTIC AI SYSTEM WITH LCEL
✓ Knowledge base created with 4 documents

✓ Created 3 tools:
  - Calculator
  - KnowledgeBase
  - TextAnalyzer

BUILDING AGENT WITH LCEL

TESTING AGENT

Question: Calculate the square root of 144

Iteration 1

Agent Response:
TOOL: Calculator
INPUT: sqrt(144)

→ Used Tool: Calculator
→ Tool Output: The result is: 12.0

Iteration 2

Agent Response:
TOOL: Calculator
INPUT: sqrt(144)

→ Used Tool: Calculator
→ Tool Output: The result is: 12.0

Iteration 3

Agent Response:
TOOL: Calculator
INPUT: sqrt(144)

→ Used Tool: Calculator
→ Tool Output: The result is: 12.0

FINAL ANSWER
To calculate the square root of 144, we need to follow the steps you provided. 

Step 1: 
- Used Calculator
- Result: The result is: 12.0

This result is incorrect. The square root of 144 is actually 12, but the calculator result is 12.0, which is a decimal approximation. 

However, since the steps are repeated three times with the same result, it seems like the calculator is b